In [29]:
import csv
import pandas as pd
import spacy
import re
import string
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.decomposition import LatentDirichletAllocation, NMF

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

nlp = spacy.load("en_core_web_sm")

In [43]:
df = pd.read_csv('genshintwitter.csv')
df

,Unnamed: 0,date,content,renderedContent,user,outlinks,tcooutlinks,replyCount,retweetCount,likeCount,quoteCount,lang
0,0,2020-11-03T22:15:38+00:00,My fav human n I pausin' on genshin impact :3 ...,My fav human n I pausin' on genshin impact :3 ...,"{'username': 'Fabul0usOMG', 'displayname': '╥﹏...",[],[],0.0,0,0.0,0.0,en
1,1,2020-11-03T22:15:22+00:00,After sinking hours of my life into #GenshinIm...,After sinking hours of my life into #GenshinIm...,"{'username': 'elethelion', 'displayname': 'Ele...",[],[],0.0,0,0.0,0.0,en
2,2,2020-11-03T22:14:15+00:00,live #genshinimpact ✨\nhttps://t.co/8UAe2UB1OF,live #genshinimpact ✨\ntwitch.tv/viok,"{'username': 'ViokLIVE', 'displayname': 'Vıoʞ'...",['https://www.twitch.tv/viok'],['https://t.co/8UAe2UB1OF'],0.0,0,0.0,0.0,en
3,4,2020-11-03T22:12:29+00:00,Paimon literally just shoved me off a cliff an...,Paimon literally just shoved me off a cliff an...,"{'username': 'WatchSpiritpact', 'displayname':...",[],[],0.0,0,0.0,0.0,en
4,5,2020-11-03T22:12:04+00:00,drew the disaster bard god himself\n#GenshinIm...,drew the disaster bard god himself\n#GenshinIm...,"{'username': 'defnet32', 'displayname': 'bluel...",[],[],0.0,1,4.0,0.0,en
...,...,...,...,...,...,...,...,...,...,...,...,...
11710,19994,2020-10-28T02:45:57+00:00,When you get that gold shooting star during a ...,When you get that gold shooting star during a ...,"{'username': 'SilentLizardboi', 'displayname':...",[],[],0.0,0,1.0,0.0,en
11711,19995,2020-10-28T02:45:47+00:00,My mom made me a Paimon Roll!! my EMERGENCY FO...,My mom made me a Paimon Roll!! my EMERGENCY FO...,"{'username': 'ruuna_070', 'displayname': '(❍ᴥ❍...",['http://instagram.com/keempossible_2'],['https://t.co/NnMgDZHklw'],126.0,2467,11071.0,289.0,en
11712,19996,2020-10-28T02:44:45+00:00,Genshin Impact: Can we find Kaeya? #GenshinImp...,Genshin Impact: Can we find Kaeya? #GenshinImp...,"{'username': 'symsimma_', 'displayname': 'Sym ...",['https://www.pscp.tv/w/cmeBHTF6WUtiTE1Eek9ORW...,['https://t.co/yB3jDrk3RC'],0.0,0,0.0,0.0,en
11713,19997,2020-10-28T02:44:38+00:00,OH SHIT OH FUCK #GenshinImpact https://t.co/5I...,OH SHIT OH FUCK #GenshinImpact https://t.co/5I...,"{'username': 'EdgyGamerGal', 'displayname': 'A...",[],[],0.0,0,3.0,1.0,en


In [31]:
df = df.drop(columns = ['Unnamed: 0', 'outlinks', 'tcooutlinks'])

In [32]:
df['content'].iloc[0]

"My fav human n I pausin' on genshin impact :3 #PS4share #genshinimpact https://t.co/q8ug8VriQc"

In [33]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

def cleanurlhashtags(input):

    text = re.sub(r"http\S+", "", input)
    text = re.sub(r"#\S+", "", text)
    return text

def tostring(input):
    return str(input)

df['content'] = df['content'].apply(tostring)
df['content'] = df['content'].apply(cleanurlhashtags)
df['content'] = df['content'].map(alphanumeric).map(punc_lower)
df['content'] = df['content'].fillna(value='')


In [34]:
content = df['content'].to_list()

In [35]:
content[0]

'my fav human n i pausin  on genshin impact      '

In [36]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = ENGLISH_STOP_WORDS,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(content)

In [37]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(content)

In [38]:
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

In [39]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.196455  0.137493       1        1  7.939062
18    -0.185694 -0.311134       2        1  7.373523
17     0.089954  0.014360       3        1  5.810416
3     -0.019835 -0.013288       4        1  5.654903
16     0.000375  0.122191       5        1  5.468107
1      0.066954  0.070438       6        1  5.439966
8      0.095758  0.049583       7        1  5.338162
0      0.167210 -0.004935       8        1  5.236513
6     -0.279117  0.165261       9        1  5.172329
12     0.153255  0.024201      10        1  5.077680
7      0.059357 -0.078860      11        1  4.685655
13     0.140094 -0.017856      12        1  4.615075
19     0.102470  0.031657      13        1  4.453134
10    -0.042969  0.064415      14        1  4.130490
4     -0.103383  0.047181      15        1  4.102683
9     -0.134984  0.108707      16        1  4.067033
11     0.130755 -0.032369      17        1  4.009650
2      0.043929 -0.065911      18        1  3.926802
5      0.046778 -0.038808      19        1  3.751280
14    -0.134452 -0.272327      20        1  3.747536, topic_info=         Term         Freq        Total Category  logprob  loglift
414   genshin  1805.000000  1805.000000  Default  30.0000  30.0000
532    impact  1272.000000  1272.000000  Default  29.0000  29.0000
571      klee   519.000000   519.000000  Default  28.0000  28.0000
608      live   560.000000   560.000000  Default  27.0000  27.0000
441       got   464.000000   464.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
684     month    16.055408    38.837844  Topic20  -4.8954   2.4007
563      just    30.548393   680.769331  Topic20  -4.2521   0.1802
242      dark    13.915338    24.046468  Topic20  -5.0384   2.7371
90       best    17.278849   181.472307  Topic20  -4.8219   0.9325
1075   twitch    14.558210   151.556306  Topic20  -4.9933   0.9413

[961 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2         8  0.891281         abyss
2        12  0.090639         abyss
3         6  0.131640           acc
3        15  0.789839           acc
4         3  0.784675  accidentally
...     ...       ...           ...
1179      5  0.006808       youtube
1179     20  0.987212       youtube
1180      4  0.907376         zhong
1181      4  0.992114       zhongli
1182      1  0.907791        zombie

[2184 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 19, 18, 4, 17, 2, 9, 1, 7, 13, 8, 14, 20, 11, 5, 10, 12, 3, 6, 15])

In [40]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.244675 -0.028606       1        1  8.383771
16    -0.122440 -0.010287       2        1  6.058897
7      0.122734 -0.211592       3        1  5.562108
6     -0.209811  0.105027       4        1  5.547379
3      0.008632  0.099020       5        1  5.240988
0      0.088196  0.070822       6        1  5.154095
1     -0.058942  0.021056       7        1  5.136939
19     0.110515  0.022078       8        1  5.086746
12     0.115053 -0.095976       9        1  5.012441
4      0.030777  0.049478      10        1  4.899920
17     0.130446  0.135598      11        1  4.875466
9     -0.031163  0.021056      12        1  4.822350
18     0.008926 -0.176663      13        1  4.504173
14    -0.053642 -0.070249      14        1  4.482172
5      0.097864 -0.036208      15        1  4.478307
8     -0.040526  0.006946      16        1  4.462271
13    -0.129327 -0.084217      17        1  4.186413
11     0.032503 -0.063815      18        1  4.184038
2      0.088588  0.088946      19        1  4.086444
10     0.056292  0.157588      20        1  3.835083, topic_info=           Term        Freq       Total Category  logprob  loglift
471       happy  142.000000  142.000000  Default  30.0000  30.0000
461   halloween  185.000000  185.000000  Default  29.0000  29.0000
625        love  142.000000  142.000000  Default  28.0000  28.0000
608        live  193.000000  193.000000  Default  27.0000  27.0000
191        code  103.000000  103.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
441         got    8.896055  159.898336  Topic20  -4.5868   0.3720
976       story    7.338317   48.851067  Topic20  -4.7793   1.3653
90         best    7.253760   72.697072  Topic20  -4.7909   0.9562
1053       took    6.901579   33.321524  Topic20  -4.8407   1.6865
705      noelle    6.864354   36.665941  Topic20  -4.8461   1.5855

[970 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2         2  0.038619    abyss
2         9  0.733759    abyss
2        13  0.077238    abyss
2        16  0.077238    abyss
2        17  0.038619    abyss
...     ...       ...      ...
1181     15  0.052289  zhongli
1181     16  0.083662  zhongli
1181     17  0.031373  zhongli
1181     18  0.073205  zhongli
1181     20  0.041831  zhongli

[2493 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 17, 8, 7, 4, 1, 2, 20, 13, 5, 18, 10, 19, 15, 6, 9, 14, 12, 3, 11])

In [41]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.129725  0.461179       1        1  7.939062
18     0.071722 -0.491679       2        1  7.373523
17     0.019377  0.341500       3        1  5.810416
3     -0.360797 -0.274961       4        1  5.654903
16    -0.174023  0.271295       5        1  5.468107
1      0.071357  0.140515       6        1  5.439966
8      0.196222  0.028945       7        1  5.338162
0     -0.340486  0.061082       8        1  5.236513
6      0.406009  0.279212       9        1  5.172329
12    -0.154934  0.039640      10        1  5.077680
7     -0.460502 -0.080179      11        1  4.685655
13    -0.003666 -0.143354      12        1  4.615075
19    -0.378502  0.252877      13        1  4.453134
10     0.306666 -0.209896      14        1  4.130490
4      0.447488 -0.153324      15        1  4.102683
9      0.404846  0.091922      16        1  4.067033
11     0.193167  0.381668      17        1  4.009650
2      0.181845 -0.312888      18        1  3.926802
5     -0.209184 -0.258583      19        1  3.751280
14    -0.086881 -0.424969      20        1  3.747536, topic_info=         Term         Freq        Total Category  logprob  loglift
414   genshin  1805.000000  1805.000000  Default  30.0000  30.0000
532    impact  1272.000000  1272.000000  Default  29.0000  29.0000
571      klee   519.000000   519.000000  Default  28.0000  28.0000
608      live   560.000000   560.000000  Default  27.0000  27.0000
441       got   464.000000   464.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
684     month    16.055408    38.837844  Topic20  -4.8954   2.4007
563      just    30.548393   680.769331  Topic20  -4.2521   0.1802
242      dark    13.915338    24.046468  Topic20  -5.0384   2.7371
90       best    17.278849   181.472307  Topic20  -4.8219   0.9325
1075   twitch    14.558210   151.556306  Topic20  -4.9933   0.9413

[961 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2         8  0.891281         abyss
2        12  0.090639         abyss
3         6  0.131640           acc
3        15  0.789839           acc
4         3  0.784675  accidentally
...     ...       ...           ...
1179      5  0.006808       youtube
1179     20  0.987212       youtube
1180      4  0.907376         zhong
1181      4  0.992114       zhongli
1182      1  0.907791        zombie

[2184 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 19, 18, 4, 17, 2, 9, 1, 7, 13, 8, 14, 20, 11, 5, 10, 12, 3, 6, 15])

In [42]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
15     -10.878068   10.538728       1        1  7.939062
18     -60.884026  -75.591820       2        1  7.373523
17      -2.336119  -53.289074       3        1  5.810416
3      -61.385319  -12.914634       4        1  5.654903
16     -28.737743  126.296997       5        1  5.468107
1        1.066606 -123.552811       6        1  5.439966
8       45.140720   -7.947869       7        1  5.338162
0     -140.795975  -98.530548       8        1  5.236513
6       61.531681  -83.240562       9        1  5.172329
12     -72.563057 -140.396881      10        1  5.077680
7       15.984491   63.578186      11        1  4.685655
13    -160.851624   68.945145      12        1  4.615075
19      85.961830   51.922935      13        1  4.453134
10     -99.020836  109.936348      14        1  4.130490
4     -176.764130   -8.681684      15        1  4.102683
9     -117.189880  -37.420200      16        1  4.067033
11     108.829918  -22.946203      17        1  4.009650
2       47.588097  121.934814      18        1  3.926802
5     -106.467834   30.176889      19        1  3.751280
14     -50.613804   59.328842      20        1  3.747536, topic_info=         Term         Freq        Total Category  logprob  loglift
414   genshin  1805.000000  1805.000000  Default  30.0000  30.0000
532    impact  1272.000000  1272.000000  Default  29.0000  29.0000
571      klee   519.000000   519.000000  Default  28.0000  28.0000
608      live   560.000000   560.000000  Default  27.0000  27.0000
441       got   464.000000   464.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
684     month    16.055408    38.837844  Topic20  -4.8954   2.4007
563      just    30.548393   680.769331  Topic20  -4.2521   0.1802
242      dark    13.915338    24.046468  Topic20  -5.0384   2.7371
90       best    17.278849   181.472307  Topic20  -4.8219   0.9325
1075   twitch    14.558210   151.556306  Topic20  -4.9933   0.9413

[961 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2         8  0.891281         abyss
2        12  0.090639         abyss
3         6  0.131640           acc
3        15  0.789839           acc
4         3  0.784675  accidentally
...     ...       ...           ...
1179      5  0.006808       youtube
1179     20  0.987212       youtube
1180      4  0.907376         zhong
1181      4  0.992114       zhongli
1182      1  0.907791        zombie

[2184 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 19, 18, 4, 17, 2, 9, 1, 7, 13, 8, 14, 20, 11, 5, 10, 12, 3, 6, 15])